# Caravan Club CLs POI Generator 2020

In [1]:
# Install any missing libraries
!pip install simplekml
!pip install pandas
!pip install folium

In [2]:
# imports
import pandas as pd
import simplekml
import folium

In [3]:
df = pd.read_csv("data/cc-cl-2020.csv", header=0)
print(df.head(5))
print(df['Address'].head())

   Longitude   Latitude                                            Address
0  -2.942832  51.263957  "Dene Hollow>01278 760520, Hill Lane, East Bre...
1  -2.972816  51.277713  "Wick Farm>01278 751229, Lympsham, Weston-supe...
2  -2.728540  50.733456  "Bridport Caravan Park>01308 424962, Silvermea...
3  -1.338463  54.217262  "The Ranch>01845 522703, Sandholme Lane, Sower...
4   1.552218  52.275819  "Green Farm>01728 668486, Darsham, Saxmundham,...
0    "Dene Hollow>01278 760520, Hill Lane, East Bre...
1    "Wick Farm>01278 751229, Lympsham, Weston-supe...
2    "Bridport Caravan Park>01308 424962, Silvermea...
3    "The Ranch>01845 522703, Sandholme Lane, Sower...
4    "Green Farm>01728 668486, Darsham, Saxmundham,...
Name: Address, dtype: object


### Extract Site Name, Telephone Number, and Address in to their own columns

In [4]:
df['SiteName'] = df['Address'].str.extract('([a-zA-Z].*(?=>))')
df['PhoneNumber'] = df['Address'].str.extract('([0-9 ]+(?=,))')
# temp data frame with split value columns 
temp = df["Address"].str.split(",", n = 1, expand = True) 

# making separate site address column from new data frame 
df["SiteAddress"]= temp[1]
    
# Dropping old Address columns 
df.drop(columns =["Address"], inplace = True) 
 
# Remove extra quote the caravan club add to the site address field!
df['SiteAddress'] = df.apply(lambda x: x['SiteAddress'][:-1], axis=1)

# df display 
df 

,Longitude,Latitude,SiteName,PhoneNumber,SiteAddress
0,-2.942832,51.263957,Dene Hollow,01278 760520,"Hill Lane, East Brent, Highbridge, Somerset, ..."
1,-2.972816,51.277713,Wick Farm,01278 751229,"Lympsham, Weston-super-Mare, Somerset, BS24 0..."
2,-2.728540,50.733456,Bridport Caravan Park,01308 424962,"Silvermead, Firch Lane, Walditch, Bridport, D..."
3,-1.338463,54.217262,The Ranch,01845 522703,"Sandholme Lane, Sowerby, Thirsk, North Yorksh..."
4,1.552218,52.275819,Green Farm,01728 668486,"Darsham, Saxmundham, Suffolk, IP17 3RN, England"
...,...,...,...,...,...
2207,-2.438378,51.800499,Passage Farm,01452 742334,"Arlingham, Gloucester, Gloucestershire, GL2 7..."
2208,-1.945363,51.376693,Court Farm,NaN,"Devizes, Wiltshire, SN10 2LD, England"
2209,-1.254455,52.824523,Littleover Farm,01509 672324,"143a Main Street, Sutton Bonington, Loughboro..."
2210,-1.359410,54.317300,Bank Head Farm,01609 780277,"Bank Head Farm, Crosby, Northallerton, North ..."


In [6]:
# Convert the phone number to a string so we can add it to folium and kml
df['PhoneNumber'] = df['PhoneNumber'].astype(str)
df['PhoneNumber']

0       01278 760520
1       01278 751229
2       01308 424962
3       01845 522703
4       01728 668486
            ...     
2207    01452 742334
2208             nan
2209    01509 672324
2210    01609 780277
2211             nan
Name: PhoneNumber, Length: 2212, dtype: object

### Now create kml file for use by maps.me
### Once the .kml file has been created for maps.me will need to edit the file and change all instances of <styleUrl>#<number></styleUrl> with a valid maps.me styleUrl
    
<styleUrl>#placemark-blue</styleUrl>
<styleUrl>#placemark-red</styleUrl>
<styleUrl>#placemark-green</styleUrl>
<styleUrl>#placemark-yellow</styleUrl>
<styleUrl>#placemark-brown</styleUrl>
<styleUrl>#placemark-orange</styleUrl>
<styleUrl>#placemark-pink</styleUrl>    
<styleUrl>#placemark-purple</styleUrl>

In [8]:
kml = simplekml.Kml()
style = simplekml.Style()
style.iconstyle.icon.href = 'http://maps.google.com/mapfiles/kml/paddle/red-blank.png'
style.iconstyle.scale = 1

map = folium.Map(location=[54.0, -1.0], zoom_start=6)
for index, row in df.iterrows():
    name = row['SiteName']
    address = row['SiteAddress']+ ', ' + row['PhoneNumber']
    phonenumber = row['PhoneNumber']
    coords = [(row['Longitude'], row['Latitude'])]
    pnt = kml.newpoint(name=name)
    pnt.coords = coords
    pnt.style = style
    pnt.phonenumber = phonenumber
    pnt.description = address
    
    popupText = folium.Html('<p>'+row['SiteName']+'</p><p>'+row['SiteAddress']+'</p><p>'+row['PhoneNumber']+'</p>', script=True)
    #popupText = folium.Html('<p>'+row['SiteName']+'</p><p>'+row['SiteAddress']+'</p>', script=True)
    popup = folium.Popup(popupText, max_width=300)
    folium.Marker([row['Latitude'], row['Longitude']],
                 popup=popup,
                 icon=folium.Icon(color='red', icon='info-sign')).add_to(map)
    
kml.save(path="poi/cc-cl-2020.kml")
map